# Predicting Low-Income Households in Selected Areas

This project aims to develop predictive models to estimate the number of low-income households in selected areas of Australia using Australian Bureau of Statistics (ABS) census data. These models will provide valuable insights for policymakers, urban planners, and social services to better understand and address socioeconomic challenges.

### Project Objectives

- Develop accurate predictive models for estimating the number of low-income households in selected areas.
- Identify key socioeconomic factors that contribute to the prevalence of low-income households.
- Create visualizations and reports to communicate findings effectively.
- Provide a tool for stakeholders to make data-driven decisions in policy and resource allocation.

### Project Scope

- Focus on predicting low-income households at the Statistical Area Level 2 (SA2) geographic level.
- Utilize ABS census data from the past census period (2021) for model development and validation.
- [WIP] Develop models for all of Australia, with the capability to focus on specific states or regions as needed.

### Data Sources

After consulting the Metadata table from General Community Profiles (GCP) of the Australian Census of 2021, 5 datasets were identified to be relevant to the project:
- Selected Medians and Averages G02
- Total Household Income (Weekly) by Household Composition G33
- Number of Motor Vehicles by Dwellings G34
- Household Composition by Number of Persons Usually Resident G35
- Mortgage Repayment (Monthly) by Dwelling Structure G38

In [34]:
# Import packagaes
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests

In [2]:
# import data

# Call csv file by name
g02 = pd.read_csv("./2021 Census GCP Statistical Area 2 for AUS/2021Census_G02_AUST_SA2.csv")
g33 = pd.read_csv("./2021 Census GCP Statistical Area 2 for AUS/2021Census_G33_AUST_SA2.csv")
g34 = pd.read_csv("./2021 Census GCP Statistical Area 2 for AUS/2021Census_G34_AUST_SA2.csv")
g35 = pd.read_csv("./2021 Census GCP Statistical Area 2 for AUS/2021Census_G35_AUST_SA2.csv")
g38 = pd.read_csv("./2021 Census GCP Statistical Area 2 for AUS/2021Census_G38_AUST_SA2.csv")

# Store DataFrames in a dictionary
gcp = {
    'g02': g02,
    'g33': g33,
    'g34': g34,
    'g35': g35,
    'g38': g38
}

## EDA

In [3]:
def shape_column(df_name, df):
    # Show dimension and column names of our data
    shape_info = f"DataFrame Name: {df_name}\nShape: {df.shape}\nColumn names: {df.columns.tolist()}"
    return print(shape_info)

In [4]:
for df_name, df in gcp.items():
    shape_column(df_name, df)

DataFrame Name: g02
Shape: (2472, 9)
Column names: ['SA2_CODE_2021', 'Median_age_persons', 'Median_mortgage_repay_monthly', 'Median_tot_prsnl_inc_weekly', 'Median_rent_weekly', 'Median_tot_fam_inc_weekly', 'Average_num_psns_per_bedroom', 'Median_tot_hhd_inc_weekly', 'Average_household_size']
DataFrame Name: g33
Shape: (2472, 61)
Column names: ['SA2_CODE_2021', 'Neg_Nil_inc_fam_households', 'Neg_Nil_inc_Non_fam_househlds', 'Negative_Nil_income_Tot', 'HI_1_149_Family_households', 'HI_1_149_Non_family_hseholds', 'HI_1_149_Tot', 'HI_150_299_Family_households', 'HI_150_299_Non_fam_househlds', 'HI_150_299_Tot', 'HI_300_399_Family_households', 'HI_300_399_Non_fam_househlds', 'HI_300_399_Tot', 'HI_400_499_Family_households', 'HI_400_499_Non_fam_househlds', 'HI_400_499_Tot', 'HI_500_649_Family_households', 'HI_500_649_Non_fam_househlds', 'HI_500_649_Tot', 'HI_650_799_Family_households', 'HI_650_799_Non_fam_househlds', 'HI_650_799_Tot', 'HI_800_999_Fam_households', 'HI_800_999_Non_fam_hsehlds', 

## Variable Selection

In [5]:
# Select columns in g02
# removing medians of variables explained in other df
g02_filtered =  g02.drop(columns=['Median_mortgage_repay_monthly',
                                 'Median_tot_prsnl_inc_weekly', 
                                 'Median_tot_fam_inc_weekly', 
                                 'Median_tot_hhd_inc_weekly'])

g02_filtered.columns.tolist()

['SA2_CODE_2021',
 'Median_age_persons',
 'Median_rent_weekly',
 'Average_num_psns_per_bedroom',
 'Average_household_size']

In [6]:
# Select columns in g33
# Select columns that contain both "HI_" and "_Tot"
g33_col_mask = g33.columns[g33.columns.str.contains("HI_") & g33.columns.str.contains("_Tot")]

# Append the column name 'SA2_CODE_2021' to the list
g33_col_mask = g33_col_mask.insert(0, 'SA2_CODE_2021')

g33_filtered = g33[g33_col_mask]

g33_filtered.columns.tolist()

['SA2_CODE_2021',
 'HI_1_149_Tot',
 'HI_150_299_Tot',
 'HI_300_399_Tot',
 'HI_400_499_Tot',
 'HI_500_649_Tot',
 'HI_650_799_Tot',
 'HI_800_999_Tot',
 'HI_1000_1249_Tot',
 'HI_1250_1499_Tot',
 'HI_1500_1749_Tot',
 'HI_1750_1999_Tot',
 'HI_2000_2499_Tot',
 'HI_2500_2999_Tot',
 'HI_3000_3499_Tot',
 'HI_3500_3999_Tot',
 'HI_4000_more_Tot']

In [7]:
# Select columns in g34
# drop total and not stated households
g34_filtered = g34.drop(columns=['Num_MVs_per_dweling_Tot', 
                                 'Num_MVs_NS', 
                                 'Total_dwelings'])

g34_filtered.columns.tolist()

['SA2_CODE_2021',
 'Num_MVs_per_dweling_0_MVs',
 'Num_MVs_per_dweling_1_MVs',
 'Num_MVs_per_dweling_2_MVs',
 'Num_MVs_per_dweling_3_MVs',
 'Num_MVs_per_dweling_4mo_MVs']

In [8]:
# Select columns in g35
# Select columns that contain "_Total"
g35_col_mask = g35.columns[g35.columns.str.contains("_Total")]

# drop the last Total column
g35_col_mask = g35_col_mask.drop('Total_Total')

# Append the column name 'SA2_CODE_2021' to the list
g35_col_mask = g35_col_mask.insert(0, 'SA2_CODE_2021')

# Apply filtering mask
g35_filtered = g35[g35_col_mask]

g35_filtered.columns.tolist()

['SA2_CODE_2021',
 'Num_Psns_UR_1_Total',
 'Num_Psns_UR_2_Total',
 'Num_Psns_UR_3_Total',
 'Num_Psns_UR_4_Total',
 'Num_Psns_UR_5_Total',
 'Num_Psns_UR_6mo_Total']

In [9]:
# Select columns in g38
# Select columns that contain both "M_" and "_Tot"
g38_col_mask = g38.columns[g38.columns.str.contains("M_") & g38.columns.str.contains("_Tot")]

# Append the column name 'SA2_CODE_2021' to the list
g38_col_mask = g38_col_mask.insert(0, 'SA2_CODE_2021')

# Apply filtering mask
g38_filtered = g38[g38_col_mask]

g38_filtered.columns.tolist()

['SA2_CODE_2021',
 'M_0_299_Tot',
 'M_300_449_Tot',
 'M_450_599_Tot',
 'M_600_799_Tot',
 'M_800_999_Tot',
 'M_1000_1399_Tot',
 'M_1400_1799_Tot',
 'M_1800_2399_Tot',
 'M_2400_2999_Tot',
 'M_3000_3999_Tot',
 'M_4000_over_Tot']

### Joining dataframes

In [10]:
def merge_dataframes(dataframes, common_column):
    # Start with the first DataFrame
    merged_df = dataframes[0]

    # Iteratively merge the remaining DataFrames
    for df in dataframes[1:]:
        merged_df = merged_df.merge(df, on=common_column, how='inner')

    return merged_df

# Merging filtered df
census_joined = merge_dataframes([g02_filtered, g33_filtered, g34_filtered, g35_filtered, g38_filtered], 'SA2_CODE_2021')
census_joined.shape

(2472, 43)

### Final data EDA

In [11]:
census_joined.dtypes

SA2_CODE_2021                     int64
Median_age_persons                int64
Median_rent_weekly                int64
Average_num_psns_per_bedroom    float64
Average_household_size          float64
HI_1_149_Tot                      int64
HI_150_299_Tot                    int64
HI_300_399_Tot                    int64
HI_400_499_Tot                    int64
HI_500_649_Tot                    int64
HI_650_799_Tot                    int64
HI_800_999_Tot                    int64
HI_1000_1249_Tot                  int64
HI_1250_1499_Tot                  int64
HI_1500_1749_Tot                  int64
HI_1750_1999_Tot                  int64
HI_2000_2499_Tot                  int64
HI_2500_2999_Tot                  int64
HI_3000_3499_Tot                  int64
HI_3500_3999_Tot                  int64
HI_4000_more_Tot                  int64
Num_MVs_per_dweling_0_MVs         int64
Num_MVs_per_dweling_1_MVs         int64
Num_MVs_per_dweling_2_MVs         int64
Num_MVs_per_dweling_3_MVs         int64


In [12]:
census_joined.isna().sum()

SA2_CODE_2021                   0
Median_age_persons              0
Median_rent_weekly              0
Average_num_psns_per_bedroom    0
Average_household_size          0
HI_1_149_Tot                    0
HI_150_299_Tot                  0
HI_300_399_Tot                  0
HI_400_499_Tot                  0
HI_500_649_Tot                  0
HI_650_799_Tot                  0
HI_800_999_Tot                  0
HI_1000_1249_Tot                0
HI_1250_1499_Tot                0
HI_1500_1749_Tot                0
HI_1750_1999_Tot                0
HI_2000_2499_Tot                0
HI_2500_2999_Tot                0
HI_3000_3499_Tot                0
HI_3500_3999_Tot                0
HI_4000_more_Tot                0
Num_MVs_per_dweling_0_MVs       0
Num_MVs_per_dweling_1_MVs       0
Num_MVs_per_dweling_2_MVs       0
Num_MVs_per_dweling_3_MVs       0
Num_MVs_per_dweling_4mo_MVs     0
Num_Psns_UR_1_Total             0
Num_Psns_UR_2_Total             0
Num_Psns_UR_3_Total             0
Num_Psns_UR_4_

## Modelling

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#!pip install xgboost
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
#import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import time
from sklearn.model_selection import GridSearchCV

In [14]:
# Prepare features and target
def prepare_features(df, target):
    features = df.columns.drop(target)
    X = df[features]
    y = df[target]
    return X, y

In [15]:
X, y = prepare_features(census_joined, 'HI_3500_3999_Tot')
X.columns

Index(['SA2_CODE_2021', 'Median_age_persons', 'Median_rent_weekly',
       'Average_num_psns_per_bedroom', 'Average_household_size',
       'HI_1_149_Tot', 'HI_150_299_Tot', 'HI_300_399_Tot', 'HI_400_499_Tot',
       'HI_500_649_Tot', 'HI_650_799_Tot', 'HI_800_999_Tot',
       'HI_1000_1249_Tot', 'HI_1250_1499_Tot', 'HI_1500_1749_Tot',
       'HI_1750_1999_Tot', 'HI_2000_2499_Tot', 'HI_2500_2999_Tot',
       'HI_3000_3499_Tot', 'HI_4000_more_Tot', 'Num_MVs_per_dweling_0_MVs',
       'Num_MVs_per_dweling_1_MVs', 'Num_MVs_per_dweling_2_MVs',
       'Num_MVs_per_dweling_3_MVs', 'Num_MVs_per_dweling_4mo_MVs',
       'Num_Psns_UR_1_Total', 'Num_Psns_UR_2_Total', 'Num_Psns_UR_3_Total',
       'Num_Psns_UR_4_Total', 'Num_Psns_UR_5_Total', 'Num_Psns_UR_6mo_Total',
       'M_0_299_Tot', 'M_300_449_Tot', 'M_450_599_Tot', 'M_600_799_Tot',
       'M_800_999_Tot', 'M_1000_1399_Tot', 'M_1400_1799_Tot',
       'M_1800_2399_Tot', 'M_2400_2999_Tot', 'M_3000_3999_Tot',
       'M_4000_over_Tot'],
      d

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training data shape:", X_train.shape, "\n" + "Validation data shape:", X_test.shape)

Training data shape: (1977, 42) 
Validation data shape: (495, 42)


In [17]:
def create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, regressor):
    # Create the pipeline with the specified regressor
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardizing the data
        ('regressor', regressor)        # The regressor passed as an argument
    ])

    # Start the timer
    start_time = time.time()

    # Fit the model
    pipeline.fit(X_train, y_train)

    # Stop the timer
    end_time = time.time()
    elapsed_time = end_time - start_time

    # Make predictions
    y_pred = pipeline.predict(X_test)

    # Evaluate the model
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Display evaluation metrics
    print(f"Regressor: {regressor.__class__.__name__}")
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")

### Open-Sourced modelling packages

In [18]:
# To use the function with different regressors:
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             RandomForestRegressor(n_estimators=100, random_state=42))
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42))
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             SVR(kernel='poly', C=1.0, epsilon=0.1))
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))

Regressor: RandomForestRegressor
Mean Absolute Error: 13.13
Mean Squared Error: 345.63
R² Score: 0.97
Time elapsed: 3.97 seconds
Regressor: GradientBoostingRegressor
Mean Absolute Error: 12.76
Mean Squared Error: 300.36
R² Score: 0.97
Time elapsed: 1.47 seconds
Regressor: ElasticNet
Mean Absolute Error: 19.05
Mean Squared Error: 672.68
R² Score: 0.94
Time elapsed: 0.01 seconds
Regressor: SVR
Mean Absolute Error: 50.23
Mean Squared Error: 4144.23
R² Score: 0.65
Time elapsed: 0.19 seconds
Regressor: XGBRegressor
Mean Absolute Error: 12.60
Mean Squared Error: 299.32
R² Score: 0.97
Time elapsed: 0.29 seconds


### SAS ML models

SAS has released a proprietary Python package named sasviya.ml that contains optimized SAS machine learning algorithms designed to run in SAS Viya Workbench.

https://go.documentation.sas.com/doc/en/workbenchcdc/v_001/explore/titlepage.htm

In [19]:
#fit a random forest model
from sasviya.ml.tree import ForestRegressor as sas_ForestRegressor
from sasviya.ml.tree import GradientBoostingRegressor as sas_GradientBoostingRegressor
from sasviya.ml.linear_model import ElasticNet as sas_ElasticNet
from sasviya.ml.svm import SVR as sas_SVR
from sasviya.ml.decomposition import PCA as sas_PCA

In [20]:
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             sas_ForestRegressor(n_estimators=100, random_state=42))
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             sas_GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             sas_ElasticNet(alpha=1.0, l1_ratio=0.5)) # no random_state parameter
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             sas_SVR(kernel='poly', C=1.0, epsilon=0.1))

Regressor: ForestRegressor
Mean Absolute Error: 14.57
Mean Squared Error: 445.34
R² Score: 0.96
Time elapsed: 0.73 seconds
Regressor: GradientBoostingRegressor
Mean Absolute Error: 13.02
Mean Squared Error: 321.14
R² Score: 0.97
Time elapsed: 0.23 seconds
Regressor: ElasticNet
Mean Absolute Error: 19.05
Mean Squared Error: 672.80
R² Score: 0.94
Time elapsed: 0.07 seconds
Regressor: SVR
Mean Absolute Error: 12.00
Mean Squared Error: 277.77
R² Score: 0.98
Time elapsed: 0.64 seconds


## Model Tuning

In [21]:
## Open source champion tuning
# Define the model
xgb = XGBRegressor(random_state=42)

# Set up the parameter grid
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__max_depth': [3, 4, 5],
    'regressor__subsample': [0.8, 1.0],
    'regressor__colsample_bytree': [0.8, 1.0]
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', xgb)
    ]),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Fit the grid search
grid_search.fit(X_train, y_train)

# Display the best parameters and best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score (neg MSE): {grid_search.best_score_}")

Fitting 3 folds for each of 72 candidates, totalling 216 fits
Best parameters: {'regressor__colsample_bytree': 0.8, 'regressor__learning_rate': 0.1, 'regressor__max_depth': 4, 'regressor__n_estimators': 200, 'regressor__subsample': 0.8}
Best cross-validation score (neg MSE): -287.39649149593185


In [22]:
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             grid_search.best_estimator_)

Regressor: Pipeline
Mean Absolute Error: 12.53
Mean Squared Error: 298.87
R² Score: 0.97
Time elapsed: 0.25 seconds


In [23]:
## SAS champion model tuning
# Specify the number of principal components to keep
n_components = 20

# using PCA to for feature selection
pca = sas_PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

pca.describe()

[Key]

               Store Key                
 
                                      Key
 A09C6B0DE18F143578D2DBDE32FECD0555536F23

[Description]

        Basic Information         
 
       Attribute              Value
 Analytic Engine                pca
    Time Created 28Nov2024:02:43:46

[InputVariables]

                           Input Variables                            
 
                         Name  Length  Role     Type RawType FormatName
                SA2_CODE_2021     8.0 Input Interval     Num           
           Median_age_persons     8.0 Input Interval     Num           
           Median_rent_weekly     8.0 Input Interval     Num           
 Average_num_psns_per_bedroom     8.0 Input Interval     Num           
       Average_household_size     8.0 Input Interval     Num           
                 HI_1_149_Tot     8.0 Input Interval     Num           
               HI_150_299_Tot     8.0 Input Interval     Num           
               HI_300_399_Tot     8.0 Input Interval     Num           
               HI_400_499_Tot     8.0 Input Interval     Num           
               HI_500_649_Tot     8.0 Input Interval     Num           
               HI_650_799_Tot     8.0 Input Interval     Num           
               HI_800_999_Tot     8.0 Input Interval     Num           
             HI_1000_1249_Tot     8.0 Input Interval     Num           
             HI_1250_1499_Tot     8.0 Input Interval     Num           
             HI_1500_1749_Tot     8.0 Input Interval     Num           
             HI_1750_1999_Tot     8.0 Input Interval     Num           
             HI_2000_2499_Tot     8.0 Input Interval     Num           
             HI_2500_2999_Tot     8.0 Input Interval     Num           
             HI_3000_3499_Tot     8.0 Input Interval     Num           
             HI_4000_more_Tot     8.0 Input Interval     Num           
    Num_MVs_per_dweling_0_MVs     8.0 Input Interval     Num           
    Num_MVs_per_dweling_1_MVs     8.0 Input Interval     Num           
    Num_MVs_per_dweling_2_MVs     8.0 Input Interval     Num           
    Num_MVs_per_dweling_3_MVs     8.0 Input Interval     Num           
  Num_MVs_per_dweling_4mo_MVs     8.0 Input Interval     Num           
          Num_Psns_UR_1_Total     8.0 Input Interval     Num           
          Num_Psns_UR_2_Total     8.0 Input Interval     Num           
          Num_Psns_UR_3_Total     8.0 Input Interval     Num           
          Num_Psns_UR_4_Total     8.0 Input Interval     Num           
          Num_Psns_UR_5_Total     8.0 Input Interval     Num           
        Num_Psns_UR_6mo_Total     8.0 Input Interval     Num           
                  M_0_299_Tot     8.0 Input Interval     Num           
                M_300_449_Tot     8.0 Input Interval     Num           
                M_450_599_Tot     8.0 Input Interval     Num           
                M_600_799_Tot     8.0 Input Interval     Num           
                M_800_999_Tot     8.0 Input Interval     Num           
              M_1000_1399_Tot     8.0 Input Interval     Num           
              M_1400_1799_Tot     8.0 Input Interval     Num           
              M_1800_2399_Tot     8.0 Input Interval     Num           
              M_2400_2999_Tot     8.0 Input Interval     Num           
              M_3000_3999_Tot     8.0 Input Interval     Num           
              M_4000_over_Tot     8.0 Input Interval     Num           

[OutputVariables]

    Output Variables     
 
   Name  Length Type Label
  Prin1     8.0  Num      
  Prin2     8.0  Num      
  Prin3     8.0  Num      
  Prin4     8.0  Num      
  Prin5     8.0  Num      
  Prin6     8.0  Num      
  Prin7     8.0  Num      
  Prin8     8.0  Num      
  Prin9     8.0  Num      
 Prin10     8.0  Num      
 Prin11     8.0  Num      
 Prin12     8.0  Num      
 Prin13     8.0  Num      
 Prin14     8.0  Num      
 Prin15     8.0  Num      
 Prin16     8.0  Num      
 Prin17   

In [24]:
# Define the SVR model
sas_svr = sas_SVR(kernel='poly', random_state=42)

# Set up the parameter grid
param_grid = {
    'regressor__C': [0.1, 1.0, 10.0],
    'regressor__epsilon': [0.01, 0.1, 0.2],
    'regressor__degree': [2, 3, 4],  # Only relevant for polynomial kernel
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', sas_svr)
    ]),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Fit the grid search
grid_search.fit(X_train_pca, y_train) # using features from PCA

# Display the best parameters and best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score (neg MSE): {grid_search.best_score_}")

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters: {'regressor__C': 10.0, 'regressor__degree': 2, 'regressor__epsilon': 0.01}
Best cross-validation score (neg MSE): -426.48878364473507


/usr/local/lib64/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
27 fits failed out of a total of 81.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
27 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib64/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib64/python3.11/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib64/python3.11/site-packages/sklearn/pipeline.py", line 420, in fit
    self._final_estimator.fit(Xt,

In [25]:
create_and_evaluate_pipeline(X_train_pca, X_test_pca, y_train, y_test, 
                             grid_search.best_estimator_)

Regressor: Pipeline
Mean Absolute Error: 12.57
Mean Squared Error: 286.90
R² Score: 0.98
Time elapsed: 0.16 seconds


In [26]:
# Define the SVR model
sas_svr = sas_SVR(kernel='poly', random_state=42)

# Set up the parameter grid
param_grid = {
    'regressor__C': [0.1, 1.0, 10.0],
    'regressor__epsilon': [0.01, 0.1, 0.2],
    'regressor__degree': [2, 3, 4],  # Only relevant for polynomial kernel
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', sas_svr)
    ]),
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1  # Use all available cores
)

# Fit the grid search
grid_search.fit(X_train, y_train) # using not selected features

# Display the best parameters and best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score (neg MSE): {grid_search.best_score_}")

Fitting 3 folds for each of 27 candidates, totalling 81 fits


2024-11-28 02:44:14,065 [ERROR   ] [151 ][_action.py     ] The total feature dimension (42) is high for polynomial degree 3.
2024-11-28 02:44:14,065 [ERROR   ] [151 ][_action.py     ] The dimension is too high to process.
2024-11-28 02:44:14,152 [ERROR   ] [151 ][_action.py     ] The total feature dimension (42) is high for polynomial degree 3.
2024-11-28 02:44:14,152 [ERROR   ] [151 ][_action.py     ] The dimension is too high to process.
2024-11-28 02:44:14,196 [ERROR   ] [151 ][_action.py     ] The total feature dimension (42) is high for polynomial degree 3.
2024-11-28 02:44:14,196 [ERROR   ] [151 ][_action.py     ] The dimension is too high to process.
2024-11-28 02:44:14,287 [ERROR   ] [151 ][_action.py     ] The total feature dimension (42) is high for polynomial degree 3.
2024-11-28 02:44:14,288 [ERROR   ] [151 ][_action.py     ] The dimension is too high to process.
2024-11-28 02:44:14,307 [ERROR   ] [151 ][_action.py     ] The total feature dimension (42) is high for polynomi

Best parameters: {'regressor__C': 0.1, 'regressor__degree': 2, 'regressor__epsilon': 0.01}
Best cross-validation score (neg MSE): -320.76405625380147


In [27]:
create_and_evaluate_pipeline(X_train, X_test, y_train, y_test, 
                             grid_search.best_estimator_)

Regressor: Pipeline
Mean Absolute Error: 12.29
Mean Squared Error: 275.26
R² Score: 0.98
Time elapsed: 0.59 seconds


Best model is the SAS SVR model before PCA. Model tuning slightly reduced the MSE and sped up compute time, but MAE increased and no imporvement to R^2.

Might see if SAS's GB model can have better performance post-tuning.

## Model Deployment

Push the model back to Viya platform for deployment and goverance thereafter.

In [28]:
name = "tztsai"

In [29]:
# best performing model
regressor = sas_SVR(kernel='poly', C=1.0, epsilon=0.1)

# Create the pipeline with the specified regressor
wb_model = Pipeline([
    ('scaler', StandardScaler()),  # Standardizing the data
    ('regressor', regressor)        # The regressor passed as an argument
])
# Fit the model
wb_model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('regressor', SVR(kernel="poly", epsilon=0.1))])

In [36]:
## get access token for viya env using refresh token. change to your own viya server and preferred authentication method.
url = "https://apgtps2demo.gtp.unx.sas.com"
auth_url = f"{url}/SASLogon/oauth/token"
## reading long-lived refresh token from txt file
refresh_token = Path(f'/workspaces/{os.environ["DEFAULT_MOUNTNAME"]}/Census_Project/apgtps2demo_refresh_token.txt').read_text().replace('\n', '')

payload=f'grant_type=refresh_token&refresh_token={refresh_token}'
headers = {
  'Accept': 'application/json',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': 'Basic c2FzLmNsaTo=',
}

response = requests.request("POST", auth_url, headers=headers, data=payload, verify=False)
access_token = response.json()['access_token']

/usr/local/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apgtps2demo.gtp.unx.sas.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyError: 'access_token'

In [ ]:
st = Session("https://apgtps2demo.gtp.unx.sas.com", token=access_token, verify_ssl=False)
st

In [ ]:
project_name = "CBR WorkshopWorkbench HandsOn "+name

In [ ]:
## Create a project
try:
    repository = mr.get_repository('DMRepository')
    # repository.name
    project_create = mr.create_project(
        {
        'name':project_name,
        'description':'CBR WB Hands-on Exercise',
        'function':'classification',
        'targetLevel':'binary',
        },
        repository
    )
    print("Created a new project!")
    project = mr.get_project(project_name)
except:
    print("Retrieving already created project")
    project = mr.get_project(project_name)

In [ ]:
def register_model(model_name, model, project):
    model_params = {
        "name": model_name,
        "projectId": project.id,
        "type": "ASTORE",
    }
    astore = mr.post(
        "/models",
        files={"files": (f"{model_name}.astore", model.export())},
        data=model_params,
    )
    print(astore)

In [ ]:
# Model Registration
register_model("MyModel "+name, wb_model, project)